# patterns



## working with trees



### walk tree?

The main feature of this is that the input type indicates the output type.



#### the code


If you query with an array, you get a list back. If you query with a string you get only the results back. Usually results are returned in a list.

If you query with an object, you get an object back with the result of each query assign to the properties of the object.



In [38]:

// returns results from multiple queries added together
function walkTree(select, ctx, evaluate) {
    var result;
    if(Array.isArray(select)) {
        result = select.reduce((arr, query, i) => {
            // pass the previous results to the next statement as the context
            if(i > 0) {
                return arr.map(r => walkTree(query, r, evaluate))
            }
            return arr.concat(walkTree(query, ctx, evaluate));
        }, []);
    } else if (typeof select === 'function') {
        // this is just here because it could be
        //   called from the array reduce above
        result = select(ctx); 
    } else if (typeof select === 'object') {
        result = Object.keys(select).reduce((obj, prop) => {
            obj[prop] = walkTree(select[prop], ctx, evaluate);
            return obj;
        }, {});
    } else {
        result = evaluate(select, ctx);
    }
    return typeof select === 'string' && Array.isArray(result)
        && result.length === 1 // TODO <= 1 undefined
       ? result[0]
       : result;
}


module.exports = {
    walkTree
}



{ walkTree: [Function: walkTree] }

#### test walking objects and lists



In [ ]:

/*

input:
[
'Item > * > Expression',
]

outut: an array of results

input:
[
'Item > * > Expression',
{
    title: '@title'
}
]

output: an array of results assign to an object with evaluate called on each
result using the result as the new context for the object evaluation

*/



### select tree?

Get all elements XPath?


#### the code



In [ ]:
var importer = require('../Core');
var {walkTree} = importer.import('walk tree');
var {XPathResult, JSDOM} = require('jsdom');
var cheerio = require('cheerio');
var assert = require('assert');

function evaluateDom(select, ctx, evaluate, query) {
    try {
    //    let $ = cheerio.load(ctx)
        if(!select.match(/^\/|\*\/|\.\//ig)) // probably XPath, fall through
            return query(select);
    } catch (e) {
        // TODO: determine any side effects of ignoring
        //if(e.name !== 'SyntaxError') {
        //    throw e;
        //}
    }
    
    try {
        // defaults to trying for iterator type
        //   so it can automatically be ordered
        var iterator = evaluate(select, ctx, null,
            ((XPathResult || {}).ORDERED_NODE_ITERATOR_TYPE || 5), null);
        // TODO: create a pattern regonizer for bodyless while
        var co = []; var m;
        while ((m = iterator.iterateNext()) && co.push(m.nodeValue || m));
        return co;
    } catch (e) {
        if(e.message.includes('Value should be a node-set')
           || e.message.includes('You should have asked')) {
            var result = evaluate(select, ctx, null,
                (XPathResult || {}).ANY_TYPE || 0, null)
            return result.resultType === ((XPathResult || {}).NUMBER_TYPE || 1)
                ? result.numberValue
                : result.resultType === ((XPathResult || {}).STRING_TYPE || 2)
                ? result.stringValue
                : result.resultType === ((XPathResult || {}).BOOLEAN_TYPE || 3)
                ? result.booleanValue
                : result.resultValue
        }
        throw e;
    }
}

// parse as html if it's string,
//   if there is no context convert the tree to html
function selectDom(select, ctx) {
    // var cheerio = require('cheerio');
    if(typeof ctx === 'string') {
        var dom = new JSDOM(ctx);
        ctx = dom.window.document;
    }
    var eval = ctx.evaluate || ctx.ownerDocument.evaluate;
    var query = ctx.querySelector || ctx.ownerDocument.querySelector
    return walkTree(select, ctx, (select, ctx) => {
        return evaluateDom(select, ctx, eval, query)
    })
}

// TODO: try catch with esquery, vm.runInThisContext, conver and select DOM, and jsel

// from least nuanced to most nuanced, CSS -> XPath -> custom ASTQ
//   Most xpath like //Element will not work on CSS, might have a problem with *
function evaluateQuery(select, ctx) {
    try {
        var esquery = require('esquery');
        // we might have to help out the CSS parser here
        if(!select.match(/^\/\/|\*\/|\.\//ig)) // probably XPath, fall through
            return esquery(ctx, select);
    } catch (e) {
        if(!e.name.includes('SyntaxError')
            && !e.message.includes('Cannot find module')) {
            throw e;
        }
    }
    
    try {
        var jsel = require('jsel');
        return jsel(ctx).selectAll(select);
    } catch (e) {
        if(!e.message.includes('XPath parse error')
          && !e.message.includes('Unexpected character')
          && !e.message.includes('Cannot find module')) {
            throw e;
        }
    }
    try {
        var ASTQ = require("astq");
        var astq = new ASTQ();
        return astq.query(ctx, select);
    } catch (e) {
        if(!e.message.includes('query parsing failed')) {
            throw e;
        }
    }
    
    throw new Error(`Could not parse select query ${JSON.stringify(select)} using XPath, CSS, or ASTQ`);
}

function selectTree(select, ctx) {
    // TODO: when converting to html, make sure to only return
    //   matching child objects not their attributes containers
    // TODO: something when we receive a string?
    //   Try to parse with all different selectors?
    return walkTree(select, ctx, evaluateQuery)
}


module.exports = {
    evaluateDom,
    evaluateQuery,
    selectTree,
    selectDom
}


#### test evaluator using code

Test tree input formats.



In [15]:

var esprima = require('esprima'); 
var escodegen = require('escodegen');
var assert = require('assert');

var code = `
var importer = require('../Core');
function name(params) {
    return importer.interpret('this is a describe request');
}
console.log()
`

function testProgram(ctx) {
    var node = selectTree('Program', ctx);
    assert(node.type === 'Program', 'just a single node not a list');
    return node;
}

function testSelect(ctx) {
    var node = selectTree([
        'Program'
    ], ctx);
    assert(node.length === 1, 'list of root node');
    assert(node[0].type === 'Program');
    // using the first query results as the context for the second query on each item
    
    var node = selectTree([
        'Identifier, CallExpression', // interchangable CSS selector
    ], ctx);
    assert(node.length > 6, 'list of concatenated elements');
    assert(typeof node[6].name !== 'undefined');
    
    var node = selectTree([
        '//*[@type="Identifier"]', // interchangable CSS selector
        './@name'
    ], ctx);
    assert(node.length > 3, 'list of ids');
    assert(node[3] === 'params');
}

module.exports = {
    testProgram,
    testSelect
}

// TODO: compare with acorn
if(typeof $$ !== 'undefined') {
    var ctx = esprima.parse(code);
    var output = testProgram(ctx);
    testSelect(ctx);
    var output = escodegen.generate(output);
    console.log(output)
}


var importer = require('../Core');
function name(params) {
    return importer.interpret('this is a describe request');
}
console.log();


#### test selecting trees with bookmarks html file

style improvements




In [2]:
var importer = require('../Core');
var getBookmarksFromTakeout = importer.import('parse bookmarks file')

if(typeof $$ !== 'undefined') {
    console.log(getBookmarksFromTakeout()[1].children[0].links)
}


[ { url: 'https://en.wikipedia.org/wiki/Point_cloud',
    title: 'Point cloud - Wikipedia',
    time_usec: 13122066361203680,
    date: 1477619659203 },
  { url: 'https://en.wikipedia.org/wiki/Kernel_method',
    title: 'Kernel method - Wikipedia',
    time_usec: 13122066367104568,
    date: 1477619665104 },
  { url: 'https://en.wikipedia.org/wiki/Support_vector_machine',
    title: 'Support vector machine - Wikipedia',
    time_usec: 13122066371037052,
    date: 1477619669037 },
  { url: 'https://en.wikipedia.org/wiki/Power_set',
    title: 'Power set - Wikipedia',
    time_usec: 13122066374266976,
    date: 1477619672266 },
  { url: 'https://en.wikipedia.org/wiki/Monte_Carlo_method',
    title: 'Monte Carlo method - Wikipedia',
    time_usec: 13122067187066330,
    date: 1477620485066 },
  { url: 'https://en.wikipedia.org/wiki/Linear_regression',
    title: 'Linear regression - Wikipedia',
    time_usec: 13122067190662088,
    date: 1477620488662 },
  { url: 'https://en.wikipedia.org

### TODO: match tree

Find matching ancestors and siblings between two trees.



## promise patterns


### resolve a promise property?

 TODO: make this even simpler, if working with promises then it can callback on it own similar to lazy loading



In [ ]:

function promiseOrResolve(obj, property, cb) {
    return obj ? (typeof obj[property] != 'undefined'
                  ? Promise.resolve(obj[property])
                  : cb(obj).then(d => (obj[property] = d)))
        : Promise.resolve()
}

module.exports = promiseOrResolve;


## classes and objects

Demonstrate patterns between different languages, in C# interfaces are like:



Long term goal: use this notebook at training data to mangle/uglify/transpile any language

```
interface MyInterface {
    myProperty: string
}

class MyClass : MyInterface {
}
```


### making a class the normal way?


In [ ]:
if(typeof parentClass === 'undefined') {
    class parentClass {
        constructor() {
            console.log('parent');
        }

        do_message() {
            console.log('original');
        }
    }

    class childClass extends parentClass {
        constructor() {
            console.log('child');
            super();
        }

        do_message() {
            super.do_message();
            console.log('overridden');
        }
    }
}

// TODO: better way to do this, childClass.prototype.bind?
module.exports = () => new childClass;

if(typeof $$ !== 'undefined') {
    console.log(typeof module.exports);
    var testClass = module.exports();
    testClass.do_message();
    
    /* expected output
    function
    child
    parent
    original
    overridden
    */
    
    /*
    this is extremely annoying, how to achieve the same effect with prototype?
    the javascript language is apparently not indempotent
    
    evalmachine.<anonymous>:1
    class parentClass {
    ^

    SyntaxError: Identifier 'parentClass' has already been declared
        at evalmachine.<anonymous>:1:1
    */
}




### extend prototype class?

derived from to help avoid annoying notebook errors:

https://stackoverflow.com/questions/10430279/extending-an-object-in-javascript


In [ ]:
var parentObj = {
    do_init() {
        console.log('parent');
    },
    do_message() {
        console.log('original');
    }
}

var childObj = {
    do_init() {
        console.log('child');
        parentObj.do_init();
    },
    do_message() {
        parentObj.do_message();
        console.log('overridden');
    }
}

function extend(child, parent) {
    var newClass = Object.create(parent || {});
    Object.assign(newClass, child, Object.getPrototypeOf(child || {}));
    var self = Object.create(newClass);
    return self;
}

function override(child) {
    return extend(child, this);
}

module.exports = {
    extend,
    override
}
//module.exports = Object.create.bind(null, childObj)

if(typeof $$ !== 'undefined') {
    console.log(typeof module.exports)
    var testClass = module.exports.extend();
    console.log(typeof testClass.do_init)
    testClass = module.exports.extend(childObj, parentObj)
    testClass.do_init();
    testClass.do_message();
    testClass = module.exports.extend({}, parentObj)
    testClass.do_init();
    testClass.do_message();
    // TODO: more tests
    
    /* expected output
    object
    undefined
    child
    parent
    original
    overridden
    parent
    original
    */
}



### enforcing an interface?


In [ ]:

// newer ES6 syntax

function typeErrorTemplate(e, k, t, i, p) {
    if(k) {
        k = ' ' + k;
    }
    if(p) {
        p = ' of type ' + p;
    }
    throw new Error(`type mis-match${k || ''}: "${t}" is not "${i}"${p || ''}`, e)
}

function standardCompare(type, expected) {
    if(type === expected) return true;
    if(!type || !expected || !(expected.isPrototypeOf(type))) {
        return false
    }
    return true;
}

function arrayCompare(compare, specification, loosey) {
    var match = specification.map(i => {
        try {
            return interface(compare, i, loosey);
        } catch (e) {
            return e;
        }
    }).filter(s => !s || s.constructor !== Error);
    if(match.length > 0) {
        return match[0];
    }
    typeErrorTemplate(void 0,
                      void 0,
                      typeof compare,
                      specification.map(s => typeof s));
}

function objectCompare(compare, specification, loosey) {
    var match = Object.keys(specification).reduce((map, k) => {
        try {
            var m = interface(compare[k], specification[k], loosey);
            if(k === 'kernel_config') {
            }
            if(typeof m !== 'undefined') {
                map[k] = m;
            }
        } catch (e) {
            typeErrorTemplate(e,
                              k,
                              typeof compare[k],
                              specification[k],
                              specification[k].constructor)
        }
        return map;
    }, {});
    return match;
}

// loosey means no exceptions are thrown and undefined type is assumed
function interface(compare, specification, loosey) {
    var type = compare === void 0 || compare === null
        ? compare
        : Object.getPrototypeOf(compare);
    var expected = specification === void 0 || specification === null
        ? specification
        : Object.getPrototypeOf(specification);
    switch(expected) {
        case Array.prototype:
            // if it is an empty array to the actual compare on the object,
            //   if it is not an empty array compare to each type in the array
            if(specification.length > 0) {
                return arrayCompare(compare, specification, loosey)
            }
        case Object.prototype:
            // compare properties on object
            if(Object.keys(specification).length > 0
               && (typeof compare === 'object' || typeof compare === 'function')) {
               return objectCompare(compare, specification, loosey)
            }
        default:
            if(standardCompare(type, expected)
               // if loosey is not explicitly set to false,
               //   return void 0 (undefined) instead of error
               //   only if compare it undefined,
               //   still error on other wrong types
               || (loosey !== false && compare === void 0)) {
                return compare;
            }
            typeErrorTemplate(void 0,
                              void 0,
                              typeof compare,
                              typeof specification,
                              expected)
    }
}

module.exports = interface;

if(typeof $$ !== 'undefined') {
    var jsonInterface = {
        display_name: '',
        argv: [], // not optional
        language: '', // not optional
        metadata: [void 0, {}], // optional
        env: [void 0, {}], // TODO dictionary descriptor types?
        interrupt_mode: [void 0, '']
    }

    console.log(interface({
        display_name: 'Node JS',
        argv: [],
        language: '',
        some_other_stuff: true
    }, jsonInterface))
    
    // expected output {"display_name":"Node JS","argv":[],"language":""}
}



#### test enforcing an interface with prototype?



In [ ]:

function myInterface(overrides) {
    var interface = Object.create({});
    // TODO: wish there was a way to not name this twice
    Object.assign(interface, {
        propertyOne: overrides.propertyOne,
        print: overrides.print
    });
    return interface;
}

function print() {
    console.log(this.propertyOne);
    console.log(this.propertyTwo);
}

var myClass = Object.create({
    propertyOne: 'original 1',
    propertyTwo: 'original 2',
    print: print
})

if(typeof $$ !== 'undefined') {

    myClass.print();
    var overridden = overrideClass({
        propertyOne: 'overridden 1',
        propertyTwo: 'overridden 2'
    })
    overridden.print();
    var interfaced = myInterface(overridden);
    interfaced.print();
    overrideClass(interfaced).print();
    myInterface(myClass).print();
    
}

/* expected output
original 1
original 2
overridden 1
overridden 2
overridden 1
undefined
overridden 1
original 2
original 1
undefined
*/



## dependency injection

Show an example of dependency injection.

Then show how context injection is so much better.

Basically a link, placeholder for a call to the languages notebook. But every interviewer asks about this crap.

